In [19]:
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Parameters
POP_SIZE = 100           # Population size
NUM_BITS = 10            # Number of bits for binary representation
GENERATIONS = 100        # Number of generations
DEFAULT_MUT_RATE = 0.1   # Default mutation rate
DEFAULT_CROSS_RATE = 0.8 # Default crossover rate

# Decode binary string to a number in a specified range
def decode(binary_string, lower_bound=0, upper_bound=1):
    decimal_value = int("".join(map(str, binary_string)), 2)
    max_decimal = 2**NUM_BITS - 1
    return lower_bound + (decimal_value / max_decimal) * (upper_bound - lower_bound)

# Fitness Functions
def fitness_x_squared(individual):
    x = decode(individual, lower_bound=1, upper_bound=50)  # Map binary to [1, 50]
    return x**2  # Fitness = x^2

def fitness_m1(individual):
    x = decode(individual)  # Map binary to [0, 1]
    return math.sin(5 * math.pi * x)**6

def fitness_m4(individual):
    x = decode(individual)  # Map binary to [0, 1]
    return math.exp(-2 * (math.log(2)) * ((x - 0.08) / 0.854)**2) * math.sin(5 * math.pi * (x**0.75 - 0.05))**6

# Initialize population with random binary strings
def initialize_pop():
    return [[random.randint(0, 1) for _ in range(NUM_BITS)] for _ in range(POP_SIZE)]

# Selection: Keep the top 50% of the population
def selection(population, fitness_function):
    fitness_values = [(ind, fitness_function(ind)) for ind in population]
    sorted_population = sorted(fitness_values, key=lambda x: x[1], reverse=True)
    return [x[0] for x in sorted_population[:POP_SIZE // 2]]

# Crossover: Single-point crossover
def crossover(parents, crossover_rate=DEFAULT_CROSS_RATE):
    offspring = []
    for _ in range(POP_SIZE):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        if random.random() < crossover_rate:
            crossover_point = random.randint(1, NUM_BITS - 1)
            child = parent1[:crossover_point] + parent2[crossover_point:]
        else:
            child = parent1
        offspring.append(child)
    return offspring

# Mutation: Flip bits with a probability of mutation_rate
def mutate(offspring, mutation_rate=DEFAULT_MUT_RATE):
    mutated_offspring = []
    for individual in offspring:
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i]  # Flip bit
        mutated_offspring.append(individual)
    return mutated_offspring

# Main Genetic Algorithm
def genetic_algorithm(fitness_function, lower_bound=0, upper_bound=1, mutation_rate=DEFAULT_MUT_RATE, crossover_rate=DEFAULT_CROSS_RATE, generations=GENERATIONS):
    population = initialize_pop()
    initial_population = population[:]

    Fmax_list, Favg_list, Fmin_list = [], [], []

    for generation in range(generations):
        fitness_values = [fitness_function(ind) for ind in population]
        Fmax = max(fitness_values)
        Favg = sum(fitness_values) / len(fitness_values)
        Fmin = min(fitness_values)
        Fmax_list.append(Fmax)
        Favg_list.append(Favg)
        Fmin_list.append(Fmin)

        parents = selection(population, fitness_function)
        offspring = crossover(parents, crossover_rate)
        population = mutate(offspring, mutation_rate)

    return initial_population, population, Fmax_list, Favg_list, Fmin_list

# Perform multiple runs and calculate average statistics
def run_multiple_experiments(fitness_function, lower_bound, upper_bound, mutation_rate, crossover_rate, runs=10, generations=GENERATIONS):
    all_Fmax, all_Favg, all_Fmin = [], [], []

    for _ in range(runs):
        _, _, Fmax_list, Favg_list, Fmin_list = genetic_algorithm(
            fitness_function, lower_bound, upper_bound, mutation_rate, crossover_rate, generations
        )
        all_Fmax.append(Fmax_list)
        all_Favg.append(Favg_list)
        all_Fmin.append(Fmin_list)

    # Average across all runs
    avg_Fmax = np.mean(all_Fmax, axis=0)
    avg_Favg = np.mean(all_Favg, axis=0)
    avg_Fmin = np.mean(all_Fmin, axis=0)

    return avg_Fmax, avg_Favg, avg_Fmin

# Visualization
def plot_fitness(Fmax_list, Favg_list, Fmin_list, title, pdf):
    plt.figure()
    plt.plot(Fmax_list, label='Fmax')
    plt.plot(Favg_list, label='Favg')
    plt.plot(Fmin_list, label='Fmin')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.legend()
    plt.title(title)
    pdf.savefig()
    plt.close()

def plot_population(initial_population, final_population, fitness_function, title, lower_bound, upper_bound, pdf):
    x = np.linspace(lower_bound, upper_bound, 1000)
    y = [fitness_function([int(b) for b in f"{int(v * (2**NUM_BITS - 1)) :010b}"]) for v in x]

    plt.figure()
    plt.plot(x, y, label='Function Landscape')
    plt.scatter([decode(ind, lower_bound, upper_bound) for ind in initial_population], [fitness_function(ind) for ind in initial_population], color='red', label='Initial Population')
    plt.scatter([decode(ind, lower_bound, upper_bound) for ind in final_population], [fitness_function(ind) for ind in final_population], color='green', label='Final Population')
    plt.xlabel('x')
    plt.ylabel('Fitness')
    plt.title(title)
    plt.legend()
    pdf.savefig()
    plt.close()

    
def run_experiments_with_landscape():
    fitness_functions = [
        (fitness_x_squared, "x^2", 1, 50),
        (fitness_m1, "M1", 0, 1),
        (fitness_m4, "M4", 0, 1)
    ]
    mutation_rates = [0.1, 0.3]
    crossover_rates = [0.5, 0.8]
    runs = 10  

    pdf_path = "GA_Experiments_Landscape.pdf"  
    with PdfPages(pdf_path) as pdf:
        for fitness_function, func_name, lb, ub in fitness_functions:
            for Pm in mutation_rates:
                for Pc in crossover_rates:
                    avg_Fmax, avg_Favg, avg_Fmin = run_multiple_experiments(
                        fitness_function, lower_bound=lb, upper_bound=ub, mutation_rate=Pm, crossover_rate=Pc, runs=runs
                    )
                    
                    
                    initial_pop, final_pop, _, _, _ = genetic_algorithm(
                        fitness_function, lower_bound=lb, upper_bound=ub, mutation_rate=Pm, crossover_rate=Pc
                    )

               
                    plot_fitness(avg_Fmax, avg_Favg, avg_Fmin, f"Averaged {func_name}: Pm={Pm}, Pc={Pc}", pdf)
                    
                    
                    plot_population(initial_pop, final_pop, fitness_function, f"{func_name} Landscape: Pm={Pm}, Pc={Pc}", lb, ub, pdf)
    return pdf_path

pdf_file_landscape = run_experiments_with_landscape()
pdf_file_landscape    

'GA_Experiments_Landscape.pdf'